# Galanov's model for mechanical properties computation

This notebook allows to compute elastic-plastic zone $\frac{b_s}{c}$, the constrain factor $C$ and ductility characteristic $\delta_H$ by solving equations proposed by Galanov *et al* (Galanov, Ivanov, et Kartuzov, *Improved Core Model of the Indentation for the Experimental Determination of Mechanical Properties of Elastic-Plastic Materials and Its Application*.)

The method is applied to mechanical properties predicted with Random Forest from [our datasets](https://zenodo.org/record/6104937#.Yg4ifC9ziRs). The originnal dataset is extended with Galanov's model computed values. 

## Import libraries 

In [1]:
import pandas as pd
import numpy as np
import os as os
import GalanovModel as gm
from scipy.optimize import fsolve


## Datasets folder

In [2]:
datasets_dir=os.getenv("DATASETS_DIR")

## Import datasets containing predictions and prepare the extension


In [3]:
dataset=pd.read_csv(f'{datasets_dir}/prediction_RF.csv', header=0,names=["Zr","Nb",'Mo',"Ti","Cr", "RF - Phase prediction from EBSD class","RF - Phase prediction from XRD class","RF - CI prediction","RF - IQ prediction",	"RF - Young Modulus prediction (GPa)","RF - Hardness prediction (GPa)"])
display(dataset)

# Create index 
nb_data=dataset.shape[0]
index_list=[]
for i in range (0,nb_data):
    index_list.append("compo_"+str(i))


dataset['index']=index_list
dataset=dataset.set_index('index')

dataset_mechanical_model=dataset.copy()

# Add columns that are going to be computed
dataset_mechanical_model['C']=np.zeros(nb_data)
dataset_mechanical_model['x']=np.zeros(nb_data)
dataset_mechanical_model['deltaH']=np.zeros(nb_data)
dataset_mechanical_model['E2/H']=np.zeros(nb_data)

Hs_column="RF - Hardness prediction (GPa)"
Es_column="RF - Young Modulus prediction (GPa)"


,Zr,Nb,Mo,Ti,Cr,RF - Phase prediction from EBSD class,RF - Phase prediction from XRD class,RF - CI prediction,RF - IQ prediction,RF - Young Modulus prediction (GPa),RF - Hardness prediction (GPa)
0,0.0,0.0,0.0,0.0,100.0,1.0,1,0.227018,57893.368438,267.790417,11.316550
1,0.0,0.0,0.0,2.0,98.0,1.0,1,0.226734,57893.368438,236.930690,11.509728
2,0.0,0.0,0.0,4.0,96.0,1.0,1,0.226929,59560.721745,235.365696,11.496725
3,0.0,0.0,0.0,6.0,94.0,1.0,1,0.228720,58880.630167,234.382639,11.411974
4,0.0,0.0,0.0,8.0,92.0,1.0,1,0.232064,59234.431667,241.943527,11.410432
...,...,...,...,...,...,...,...,...,...,...,...
316246,98.0,0.0,0.0,0.0,2.0,0.0,1,0.019261,36816.220090,126.529137,7.808660
316247,98.0,0.0,0.0,2.0,0.0,0.0,1,0.019261,36816.220090,126.529137,7.808660
316248,98.0,0.0,2.0,0.0,0.0,0.0,1,0.019261,36816.220090,126.529137,7.808660
316249,98.0,2.0,0.0,0.0,0.0,0.0,1,0.019261,36816.220090,126.529137,7.808660


## Initialize Galanov's model constants and solve the system

You will need:
$E_s$, $nu_s$, $H_s$ : elastic modulus, Poisson ratio and hardness of the sample
$E_i$, $nu_i$: elastic modulus and Poisson ratio of the tip

Other constants are computed with `gm.Galanov_math_values`. 

Galanov's system is solved with `gm.system_x_y`, y being $\frac{1}{C}$ and x being $\frac{b_s}{c}$. We give several starting points to solve the system : some starting points do not allow the solver to converge. We must check that the solution are coherent: the intervals for y and x are taken from Galanov's paper results. We then check that the solutions obtained from the differetn starting points are equals. In this case we add the compted values to the dataset and we save the extended dataset.


In [4]:
nu_s=0.3
nu_i=0.07
E_i=1141

for j in range (0,nb_data):   
    H_s=dataset[Hs_column]['compo_'+str(j)]
    E_s=dataset[Es_column]['compo_'+str(j)]

    [Ei_star,Ks,alpha_s,beta_s,theta_s,cot_gamma_i,z]=gm.Galanov_math_values(E_s,nu_s,E_i,nu_i,H_s)

    # starting points for system solving
    starting_x_y=[[1,1],[2,0.5],[3,0.3],[5,0.25]]

    #initialize list of solution obtained from each starting points
    x_sol_list=[]
    y_sol_list=[]
    
    # for each starting point
    for i in range (0,4):
        # solve solution
        [x_sol, y_sol] = fsolve(gm.system_x_y, starting_x_y[i],args=(alpha_s,beta_s,theta_s,z))

       # Check that the computed values verify the system and that they are in range of Galanov given values
        if np.isclose(gm.system_x_y([x_sol,y_sol],alpha_s,beta_s,theta_s,z), [0.0, 0.0]).all()==True and 1/y_sol<5 and 1/y_sol>0.5 and x_sol<7 and x_sol > 0.5:
            x_sol_list.append(x_sol)
            y_sol_list.append(y_sol)
    
   # if no solution was found
    if len(x_sol_list)==0:
        C=0
        x=0
    else:
        # if the solutions found from the different strating points are equals
        if np.std(x_sol_list)<10**(-8) and np.std(y_sol_list)<10**(-8) :
            C=1/np.mean(y_sol_list)
            x=np.mean(x_sol_list)
        else:
            print('no unique solution')
            
    
    delta_H=gm.delta_H_value(E_s,H_s,nu_s,z)

    dataset_mechanical_model['C']['compo_'+str(j)]=C
    dataset_mechanical_model['x']['compo_'+str(j)]=x
    dataset_mechanical_model['deltaH']['compo_'+str(j)]=delta_H
    dataset_mechanical_model['E2/H']['compo_'+str(j)]=E_s**2/H_s

dataset_mechanical_model.to_csv('./prediction_RF_mechanical_model.csv')

a:\PUBLI_CONF\PUBLICATION\Fichier git\accelerated-exploration-of-multinary\GalanovModel\GalanovModel.py:130: RuntimeWarning: invalid value encountered in log
  eq2=1/y-(2/3+y*(x**3-alpha_s)/beta_s*np.log((1-y*(x**3-alpha_s)/(2*beta_s))**(-1))+2*np.log(x))
C:\Users\boichot\Anaconda3\envs\machinelearning\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\boichot\AppData\Local\Temp/ipykernel_31388/3885549303.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_mechanical_model['C']['compo_'+str(j)]=C
C:\Users\boichot\AppData\Local\Temp/ipykernel_31388/3885549303.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of